In [5]:
# main.ipynb

%pip install nltk
%pip install scikit-learn

import pandas as pd
import numpy as np
import string
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

# Download NLTK resources
nltk.download('stopwords')
from nltk.corpus import stopwords


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\d7144\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import pandas as pd

# loading both csv files (path goes up one level first)
fake = pd.read_csv('../dataset/Fake.csv')
real = pd.read_csv('../dataset/True.csv')

# adding labels: 0 = Fake, 1 = Real
fake['label'] = 0
real['label'] = 1

# combining both datasets and keeping only text + label
data = pd.concat([fake, real])
data = data[['text', 'label']]
data = data.sample(frac=1).reset_index(drop=True)

# previewing the result
data.head()


,text,label
0,Tune in to the Alternate Current Radio Network...,0
1,He may not have been President Trump s biggest...,0
2,The new cold war between Saudi Arabia and Iran...,0
3,Please RT until we identify this disgusting N...,0
4,"On Tuesday, Bernie Sanders slammed New York s ...",0


In [15]:
import re

# cleaning text: making lowercase, removing links, special characters, and extra spaces
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# applying the clean_text function to each row
data['clean_text'] = data['text'].apply(clean_text)

# checking cleaned output
data[['text', 'clean_text']].head()


,text,clean_text
0,Tune in to the Alternate Current Radio Network...,tune in to the alternate current radio network...
1,He may not have been President Trump s biggest...,he may not have been president trump s biggest...
2,The new cold war between Saudi Arabia and Iran...,the new cold war between saudi arabia and iran...
3,Please RT until we identify this disgusting N...,please rt until we identify this disgusting ny...
4,"On Tuesday, Bernie Sanders slammed New York s ...",on tuesday bernie sanders slammed new york s c...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# turning cleaned text into TF-IDF vectors
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X = tfidf.fit_transform(data['clean_text'])

# labels are already stored in the 'label' column
y = data['label']

# splitting data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# fitting logistic regression on the training data
model = LogisticRegression()
model.fit(X_train, y_train)

# making predictions on the test set
y_pred = model.predict(X_test)

# checking overall performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9867483296213808

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4663
           1       0.99      0.99      0.99      4317

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980


Confusion Matrix:
 [[4602   61]
 [  58 4259]]


In [19]:
import pickle
# saving the trained logistic regression model
with open('../model/news_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# saving the TF-IDF vectorizer as well
with open('../model/tfidf.pkl', 'wb') as tfidf_file:
    pickle.dump(tfidf, tfidf_file)
